In [6]:
import numpy as np
import nolds
from scipy import interpolate
from scipy import signal
import pandas as pd

In [7]:
def clean_outlier(RrIntervals, removing_rule = 0.2):
    """
    RR intervals differing by more than the removing_rule from the one proceeding 
    it are removed.
    
    Arguments
    ---------
    RrIntervals - list of Rr Intervals
    removing_rule - percentage criteria of difference with previous Rr Interval at which we
    consider that it is abnormal
    
    Returns
    ---------
    NNs - list of NN Interval

    """ 
    NNs = []
    outlier_count = 0
    for i, rri in enumerate(RrIntervals):        
        if abs(rri - RrIntervals[i-1]) <= removing_rule * RrIntervals[i-1]:
            NNs.append(rri)
        else:
            outlier_count += 1
    print("Il y a {} outliers qui ont été supprimés".format(outlier_count))
    return NNs

# Timeserie test
index_test = pd.date_range(start = "09/07/2018", periods = 24*60*60 ,freq = "s")
ts_test = pd.Series(np.random.randint(600, 800, 24*60*60), index=index_test)

clean_outlier(ts_test[:300])

Il y a 27 outliers qui ont été supprimés


[686,
 751,
 779,
 788,
 749,
 692,
 692,
 734,
 773,
 653,
 686,
 691,
 611,
 654,
 692,
 703,
 695,
 702,
 752,
 615,
 682,
 765,
 618,
 671,
 697,
 731,
 726,
 727,
 633,
 634,
 629,
 740,
 699,
 603,
 655,
 657,
 752,
 723,
 643,
 628,
 706,
 754,
 735,
 608,
 671,
 652,
 691,
 745,
 722,
 790,
 795,
 753,
 603,
 684,
 740,
 674,
 797,
 652,
 723,
 652,
 682,
 652,
 703,
 656,
 756,
 663,
 741,
 775,
 715,
 789,
 718,
 738,
 607,
 628,
 657,
 669,
 783,
 658,
 637,
 658,
 760,
 627,
 674,
 758,
 789,
 780,
 799,
 708,
 786,
 644,
 710,
 683,
 656,
 642,
 624,
 689,
 688,
 675,
 797,
 798,
 732,
 733,
 699,
 646,
 746,
 661,
 624,
 617,
 679,
 723,
 766,
 777,
 613,
 704,
 646,
 759,
 788,
 702,
 692,
 662,
 618,
 694,
 724,
 732,
 758,
 612,
 668,
 643,
 697,
 698,
 649,
 628,
 680,
 772,
 782,
 682,
 646,
 747,
 673,
 604,
 686,
 645,
 628,
 651,
 621,
 665,
 670,
 689,
 759,
 618,
 674,
 660,
 697,
 739,
 612,
 612,
 642,
 603,
 694,
 789,
 681,
 625,
 672,
 693,
 656,
 651,
 602

In [8]:
def get_time_domain_features(NNIntervals):
    """
    Function returning a dictionnary containing time domain features for 
    HRV analyses.
    Must use this function on short term recordings, from 2 to 5 minutes 
    window.
    
    Arguments
    ----------
    NNIntervals - list of Normal to Normal Interval
    
    Returns
    ----------
    timeDomainFeaturess - dictionnary containing time domain features for 
    HRV analyses. Thera are details about each features below.
    
    Notes
    ----------
    Details about feature engineering...
    
    - **meanNN**: The the mean RR interval
    
    - **SDANN**: The standard deviation of the time interval between successive normal heart 
    beats (*i.e.*, the RR intervals)
    
    - **SDSD**: Standard deviation of differences between adjacent NN intervals
    
    - **RMSSD**: Root mean square of the RR intervals (*i.e.*, square root of the mean 
    of the squared differences in time between successive normal heart beats). Reflects high 
    frequency (fast or parasympathetic) influences on HRV (*i.e.*, those influencing larger 
    changes from one beat to the next).
    
    - **medianNN**: Median Absolute values of the successive Differences between the RR intervals.

    - **NN50**: Number of interval differences of successive RR intervals greater than 50 ms.

    - **pNN50**: The proportion derived by dividing NN50 (The number of interval differences of 
    successive RR intervals greater than 50 ms) by the total number of RR intervals.

    - **NN20**: Number of interval differences of successive RR intervals greater than 20 ms.

    - **pNN20**: The proportion derived by dividing NN20 (The number of interval differences of 
    successive RR intervals greater than 20 ms) by the total number of RR intervals.

    - **CVSD**: The coefficient of variation of successive differences (van Dellen et al., 1985), 
    the RMSSD divided by meanNN.
    
    - **RangeNN**: différence between the maximum and minimum NNInterval.
    
    
    References
    ----------
    TO DO
    
    """
    
    diff_nni = np.diff(NNIntervals)
    L = len(NNIntervals)
    
    meanNN = np.mean(NNIntervals)
    SDANN = np.std(NNIntervals, ddof=1) # ddof = 1 : estimateur non biaisé => divise std par n-1
    SDSD = np.std(diff_nni)
    RMSSD = np.sqrt(np.mean(diff_nni ** 2))
    medianNN = np.median(NNIntervals)
    
    NN50 = sum(abs(diff_nni) > 50)
    pNN50 = 100 * NN50 / L
    NN20 = sum(abs(diff_nni) > 20)
    pNN20 = 100 * NN20 / L
    
    RangeNN = max(NNIntervals) - min(NNIntervals)
    
    # Feature(s) trouvée(s) sur les codes github et non dans la doc
    #CVSD = RMMSD / meanNN
    
    # Features non calculables pour short term recordings
    #SDNN équivaut à SDANN sur 24h et non sur une fenêtre temporelle
    #cvNN = SDNN / meanNN # Necissite cycle 24h
    
    timeDomainFeaturess = {
        'meanNN': meanNN, 
        'SDANN': SDANN, 
        'SDSD': SDSD, 
        'NN50': NN50,
        'pNN50' : pNN50, 
        'NN20' : NN20, 
        'pNN20' : pNN20, 
        'RMSSD' : RMSSD,
        'MedianNN' : medianNN, 
        'RangeNN' : RangeNN
        #,'CVSD' : CVSD,
        #'SDNN' : SDNN,
        #'cvNN' : cvNN
        
    }
    
    return timeDomainFeaturess

# TO DO 
# GEOMETRICAL FEATURES
# Timeserie test
index_test = pd.date_range(start = "09/07/2018", periods = 24*60*60 ,freq = "s")
ts_test = pd.Series(np.random.randint(600, 800, 24*60*60), index=index_test)

get_time_domain_features(ts_test[:300])

{'meanNN': 700.03,
 'SDANN': 57.19411089500591,
 'SDSD': 75.72775707329069,
 'NN50': 156,
 'pNN50': 52.0,
 'NN20': 238,
 'pNN20': 79.33333333333333,
 'RMSSD': 75.72863452457413,
 'MedianNN': 697.0,
 'RangeNN': 199}

In [9]:
def get_frequency_domain_features(NNIntervals, sampling_frequency = 8, interpolation_method = "linear", 
                                 VLF_band = [0.0033, 0.04], LF_band = [0.04, 0.15], HF_band = [0.15, 0.40]):
    """
    Function returning a dictionnary containing frequency domain features 
    for HRV analyses.
    Must use this function on short term recordings, from 2 to 5 minutes 
    window.
    
    Arguments
    ---------
    NNIntervals - list of Normal to Normal Interval
    sampling_frequency - frequence at which the signal is sampled
    interpolation_method - kind of interpolation as a string, by default "linear"
    VLF_band - Very low frequency band for features extraction from power spectral density
    LF_band - Low frequency band for features extraction from power spectral density
    HF_band - High frequency band for features extraction from power spectral density
    
    Returns
    ---------
    freqency_domain_features - dictionnary containing frequency domain features 
    for HRV analyses. Thera are details about each features are given in 
    "get_features_from_PSD" function.
    
    References
    ----------
    TO DO
    """

    # ---------- Interpolation du signal ---------- #
    
    timestamps = create_time_info(NNIntervals)
    function = interpolate.interp1d(x = timestamps, y = NNIntervals, 
                             kind = interpolation_method)
    
    timestamps_interpolation = create_interpolation_time(NNIntervals, sampling_frequency)
    NNI_interpolation = function(timestamps_interpolation)
    
    #Remove DC component --> A VERIFIER DANS DOCS BIBLIO  
    # Annabelle OK pour remove DC comp
    RRseries = NNI_interpolation - np.mean(NNI_interpolation)
    
    #  ----------  Calcul PSD  ---------- #
    freq, Pxx = signal.welch(x = RRseries, fs = sampling_frequency,
                            window = 'hann')    

    # ----------  Calcul des features  ---------- #
    freqency_domain_features = get_features_from_PSD(freq = freq, Pxx = Pxx,
                                                     VLF_band = VLF_band,
                                                     LF_band = LF_band,
                                                     HF_band = HF_band)
    
    return freqency_domain_features

def create_time_info(NNIntervals):
    """
    TO DO
    """
    # Convert in seconds
    NNI_tmstp = np.cumsum(NNIntervals) / 1000.0 
    
    # Force to start at 0
    return NNI_tmstp - NNI_tmstp[0]

def create_interpolation_time(NNIntervals, sampling_frequency = 8):
    """
    TO DO
    """
    time_NNI = create_time_info(NNIntervals)
    # Create timestamp for interpolation
    return np.arange(0, time_NNI[-1], 1 / float(sampling_frequency))


def get_features_from_PSD(freq, Pxx, VLF_band = [0.0033, 0.04], 
                              LF_band = [0.04, 0.15], 
                              HF_band = [0.15, 0.40]): #, ULF_band):
    """
    Function computing frequency domain features from the power spectral 
    decomposition.
    
    Arguments
    ---------
    to do
    
    Returns
    ---------
    to do
    
    Notes
    ---------
    Details about feature engineering...
    
    - **total_power** :
    - **VLF** :
    - **LF** : 
    - **HF** :
    - **LF_HF_ratio** : 
    - **LFnu** : 
    - **HFnu** :
    
    References
    ---------
    TO DO
    
    """
    
    # Calcul des indices selon les bandes de fréquences souhaitées
    VLF_indexes = np.logical_and(freq >= VLF_band[0], freq < VLF_band[1])
    LF_indexes = np.logical_and(freq >= LF_band[0], freq < LF_band[1])
    HF_indexes = np.logical_and(freq >= HF_band[0], freq < HF_band[1])

    # STANDARDS

    # Integrate using the composite trapezoidal rule
    VLF = np.trapz(y=Pxx[VLF_indexes], x=freq[VLF_indexes])
    LF = np.trapz(y=Pxx[LF_indexes], x=freq[LF_indexes])
    HF = np.trapz(y=Pxx[HF_indexes], x=freq[HF_indexes])

    # Feature utilisée plus souvent dans les analyses "Long term recordings"
    total_power = VLF + LF + HF
    
    LF_HR_ratio = LF / HF
    LFnu = (LF / (total_power - VLF)) * 100
    HFnu = (HF / (total_power - VLF)) * 100

    
    # Features non calculables pour "short term recordings"
    #ULF = np.trapz(y=Pxx[ULF_indexes], x=freq[ULF_indexes])

    # Feature(s) trouvée(s) sur les codes github et non dans la doc
    #LFn = LF / (LF + HF)
    #HFn = HF /(LF + HF)
    #LF_P_ratio = LF / total_power
    #HF_P_ratio = HF/ total_power

    freqency_domain_features = {
        'total_power' : total_power,
        'VLF' : VLF,
        'LF' : LF,
        'HF' : HF,
        'LF_HR_ratio' : LF_HR_ratio,
        'LFnu' : LFnu,
        'HFnu' : HFnu
        #,'ULF' : ULF,
        #'LFn': LFn,
        #'HFn' : HFn,
        #'LF_P_ratio' : LF_P_ratio,
        #'HF_P_ratio' : HF_P_ratio
    }
    
    return freqency_domain_features

# Timeserie test
index_test = pd.date_range(start = "09/07/2018", periods = 24*60*60 ,freq = "s")
ts_test = pd.Series(np.random.randint(600, 800, 24*60*60), index=index_test)

get_frequency_domain_features(ts_test[:300])

{'total_power': 990.9958079097867,
 'VLF': 0.0,
 'LF': 198.78026349084035,
 'HF': 792.2155444189464,
 'LF_HR_ratio': 0.25091689362979686,
 'LFnu': 20.058638180327794,
 'HFnu': 79.94136181967221}

In [10]:
def get_csi_cvi_features(NNIntervals):
    """
    TO DO Décrire fonction
    Must use this function on short term recordings, for 30 , 50, 100 Rr Intervals 
    or seconds window.

    Arguments
    ---------
    TO DO
    
    Returns
    ---------
    TO DO
    
    Notes
    ---------
    TO DO
    
    References
    ----------
    TO DO    
    """
    
    # Measures the width and length of poincare cloud
    poincare_plot_features = get_poincare_plot_features(NNIntervals)
    T = 4 * poincare_plot_features['SD1']
    L = 4 * poincare_plot_features['SD2']
    
    CSI = L / T
    CVI = np.log10(L * T)
    Modified_CSI = L**2 / T
    
    csi_cvi_features = {
        'CSI' : CSI,
        'CVI' : CVI,
        'Modified_CSI' : Modified_CSI
    }
    
    return csi_cvi_features

def get_poincare_plot_features(NNIntervals):
    """
    TO DO
    window 5 min
    """
    diff_NNIntervals = np.diff(NNIntervals)
    #measures the width of poincare cloud
    SD1 = np.sqrt(np.std(diff_NNIntervals, ddof=1) ** 2 * 0.5)
    #measures the length of the poincare cloud
    SD2 = np.sqrt(2 * np.std(NNIntervals, ddof=1) ** 2 - 0.5 * np.std(diff_NNIntervals,
                                                              ddof=1) ** 2)
    ratio_SD1_SD2 = SD1 / SD2
    
    poincare_plot_features = {
        'SD1' : SD1,
        'SD2' : SD2,
        'ratio_SD1_SD2' : ratio_SD1_SD2
    }

    return poincare_plot_features

def get_sampen(NNIntervals):
    """
    window : 1 min
    TO DO
    """
    SampEn = nolds.sampen(NNIntervals, emb_dim = 2)
    return {'Sampen' : SampEn}

# Timeserie test
index_test = pd.date_range(start = "09/07/2018", periods = 24*60*60 ,freq = "s")
ts_test = pd.Series(np.random.randint(600, 800, 24*60*60), index=index_test)

print(get_csi_cvi_features(ts_test[:300]))
print()
print(get_poincare_plot_features(ts_test[:300]))
print()
print(get_sampen(ts_test[:300]))

{'CSI': 1.1087548999552461, 'CVI': 4.7484844879800905, 'Modified_CSI': 276.37297416891687}

{'SD1': 56.20364430359361, 'SD2': 62.316066016951176, 'ratio_SD1_SD2': 0.9019125868488735}

{'Sampen': 2.172223275130802}


# Data Test

In [7]:
import json

# Load data example 
with open('data_example/2b3d7a98-a1fb-47ad-9705-4a91d448da9b_RrInterval_2018-07-03T160210842.json') as f:
    data = json.load(f)
data

{'user': '2b3d7a98-a1fb-47ad-9705-4a91d448da9b',
 'type': 'RrInterval',
 'device_address': 'CC:AE:96:E7:D6:AA',
 'data': ['2018-07-03T16:02:10.842 736',
  '2018-07-03T16:02:11.800 730',
  '2018-07-03T16:02:12.774 719',
  '2018-07-03T16:02:13.773 704',
  '2018-07-03T16:02:14.725 712',
  '2018-07-03T16:02:15.709 718',
  '2018-07-03T16:02:16.686 724',
  '2018-07-03T16:02:17.649 734',
  '2018-07-03T16:02:18.625 744',
  '2018-07-03T16:02:19.599 759',
  '2018-07-03T16:02:20.592 756',
  '2018-07-03T16:02:21.573 732',
  '2018-07-03T16:02:22.558 743',
  '2018-07-03T16:02:23.501 744',
  '2018-07-03T16:02:24.481 740',
  '2018-07-03T16:02:25.449 741',
  '2018-07-03T16:02:26.428 754',
  '2018-07-03T16:02:27.886 751',
  '2018-07-03T16:02:28.862 755',
  '2018-07-03T16:02:29.842 772',
  '2018-07-03T16:02:31.787 782',
  '2018-07-03T16:02:31.788 782',
  '2018-07-03T16:02:32.784 789',
  '2018-07-03T16:02:33.745 779',
  '2018-07-03T16:02:34.718 780',
  '2018-07-03T16:02:35.687 760',
  '2018-07-03T16:02:36

In [10]:
import pandas as pd

data_field = data["data"]
# Split fields in timestamp & value
df_exploration = pd.DataFrame(list(map(lambda x: x.split(" "), data_field)), columns= ["timestamp", "RrInterval"])

# Cast value as int
df_exploration["RrInterval"] = df_exploration["RrInterval"].apply(lambda x : int(x))

# set cleaned timestamp as index
df_exploration["timestamp"] = pd.to_datetime(df_exploration["timestamp"])
df_exploration = df_exploration.set_index("timestamp")

df_exploration.sample(5)

,RrInterval
timestamp,
2018-07-03 16:10:22.719,746
2018-07-03 16:05:36.559,676
2018-07-03 16:15:43.495,702
2018-07-03 16:17:48.788,808
2018-07-03 16:07:59.388,755
